#### This Notebook use Doc AI and converts all PDF document to text so that down the line process can work

#### Author: Saurabh Mangal (saurabhmangal@google.com)
#### Editor / Reviewer: Wan Qi, Jing Le
##### Date: 21st Feb
##### Description: This notebook contains part 1 of lab

 Copyright (c) [2024] [saurabhmangal@] -- 
 This notebook is licensed under the Commercial License.

In [1]:
!pip install --quiet google-cloud-discoveryengine
!pip install --upgrade --quiet google-cloud-storage
%pip install --quiet google-cloud-discoveryengine
%pip install --upgrade --quiet google-cloud-storage

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Please enter your name/initials (no spaces or special characters allowed), ensure that it is unique
UNIQUE_PREFIX="singhealth"

In [3]:
import re

PROJECT_IDS = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_IDS[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

In [4]:
from google.cloud import storage

client = storage.Client()

GCS_BUCKET_LOCATION = "asia-southeast1"

GCS_BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}"
GCS_BUCKET_URI = f"gs://{GCS_BUCKET_NAME}"

bucket = storage.Bucket(client, GCS_BUCKET_NAME)

if bucket.exists()==False:
    # Create a Cloud Storage Bucket
    !gcloud storage buckets create $GCS_BUCKET_URI --location=$GCS_BUCKET_LOCATION

    # Upload the PDFs located in the books/ directory into the GCS bucket that you created
    !gsutil cp -r $FOLDER_NAME/books/* $GCS_BUCKET_URI

    # Verify that all Books 1 to 7 are uploaded to the GCS bucket (8 files in total, 2 for Part 1)
    !gsutil ls $GCS_BUCKET_URI
else:
    # Upload the PDFs located in the books/ directory into the GCS bucket that you created
    !gsutil cp -r $FOLDER_NAME/books/* $GCS_BUCKET_URI

    # Verify that all Books 1 to 7 are uploaded to the GCS bucket (8 files in total, 2 for Part 1)
    !gsutil ls $GCS_BUCKET_URI
    
    print(f"{GCS_BUCKET_NAME} already exists. Please proceed as usual.")

Creating gs://my-project-0004-346516-singhealth/...
Copying file://./genai_03/books/Book1_HarryPotter_and_the_Sorcerers_Stone_pg15.pdf [Content-Type=application/pdf]...
Copying file://./genai_03/books/Winnie_the_Pooh_3_Pages.pdf [Content-Type=application/pdf]...
| [2 files][643.2 KiB/643.2 KiB]                                                
Operation completed over 2 objects/643.2 KiB.                                    
gs://my-project-0004-346516-singhealth/Book1_HarryPotter_and_the_Sorcerers_Stone_pg15.pdf
gs://my-project-0004-346516-singhealth/Winnie_the_Pooh_3_Pages.pdf


In [5]:
from typing import Optional

from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine


#Must specify either `gcs_uri` or (`bigquery_dataset` and `bigquery_table`)
#Format: `gs://bucket/directory/object.json` or `gs://bucket/directory/*.json`
gcs_uri = GCS_BUCKET_URI


def import_documents_sample(
    project_id: str,
    location: str,
    data_store_id: str,
    gcs_uri: Optional[str] = None,
    bigquery_dataset: Optional[str] = None,
    bigquery_table: Optional[str] = None,
) -> str:
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.DocumentServiceClient(client_options=client_options)

    # The full resource name of the search engine branch.
    # e.g. projects/{project}/locations/{location}/dataStores/{data_store_id}/branches/{branch}
    parent = client.branch_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        branch="default_branch",
    )

    if gcs_uri:
        request = discoveryengine.ImportDocumentsRequest(
            parent=parent,
            gcs_source=discoveryengine.GcsSource(
                input_uris=[gcs_uri], data_schema="custom"
            ),
            # Options: `FULL`, `INCREMENTAL`
            reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
        )
    else:
        request = discoveryengine.ImportDocumentsRequest(
            parent=parent,
            bigquery_source=discoveryengine.BigQuerySource(
                project_id=project_id,
                dataset_id=bigquery_dataset,
                table_id=bigquery_table,
                data_schema="custom",
            ),
            # Options: `FULL`, `INCREMENTAL`
            reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
        )

    # Make the request
    operation = client.import_documents(request=request)

    print(f"Waiting for operation to complete: {operation.operation.name}")
    response = operation.result()

    # Once the operation is complete,
    # get information from operation metadata
    metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)

    # Handle the response
    print(response)
    print(metadata)

    return operation.operation.name


#### We try both open source pdf option as well as DOC AI 

In [6]:
# !pip install PyMuPDF Pillow pytesseract

In [7]:
# !pip install langchain
!gcloud services enable bigquery.googleapis.com

In [8]:
%pip install --quiet PyPDF2
%pip install --quiet pdfreader

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/python_dateutil-2.8.2.dist-info/METADATA'

Note: you may need to restart the kernel to use updated packages.


In [9]:
from pdfreader import PDFDocument, SimplePDFViewer

In [10]:
# Load the PDF document
pdf_file = f"{FOLDER_NAME}/books/Book1_HarryPotter_and_the_Sorcerers_Stone_pg15.pdf"

In [11]:
fd = open(pdf_file, "rb")
doc = PDFDocument(fd)

In [12]:
from io import BytesIO
with open(pdf_file, "rb") as f:
    stream = BytesIO(f.read())
doc2 = PDFDocument(stream)

In [13]:
page_one = next(doc.pages())

In [14]:
# Extract text from the page
page_one.Contents

<Stream:len=3699,data=b'x\x9c\xdd]K\x8b\xe4\xc8\x11\xbe\xf7\xaf\xd0\xd9`\xad\xdeR\x811\xcc\xf4\xce\x18| ...'>

In [15]:
all_pages = [p for p in doc.pages()]
len(all_pages)

15

### Doc ai - Easiest and faster solution for all PDF documents 

In [16]:
!pip3 install --upgrade -q google-cloud-documentai
!pip3 install --upgrade -q google-cloud-storage
!pip3 install --upgrade -q google-cloud-documentai-toolbox

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/python_dateutil-2.8.2.dist-info/METADATA'



In [17]:
!ls

 20230203_alphabet_10K.pdf
'Advanced_RAG_Strategies_w_Langchain_ipynb_(go_adv_rag_colab).ipynb'
 Course
 Course_v2.zip
 Drain_detection_trip_30sec.mov
 PXL_20240220_113209161.mpeg
 README.md
 Trip_Itinerary_Planner_using_Vertex_LLM.ipynb
'Vertex_AI_Gemini_PRO_Testing (1).ipynb'
 Vertex_AI_Gemini_PRO_Testing.ipynb
 Vertex_AI_Gemini_PRO_Testing_v2.ipynb
'Vertex_LLM_Factuality_fishfood (1).ipynb'
 books
 eval-gen-ai
 genai_03
 generative-ai
 google_cloud_aiplatform-1.38.dev20231206+generative.models-py2.py3-none-any.whl
 google_cloud_aiplatform-1.44.dev20240308+rapid.genai.evaluation-py2.py3-none-any.whl
 hpc-toolkit
 istockphoto-479669340-640_adpp_is.mp4
 matchingengine
 medical_data.csv
'model_garden_pytorch_falcon_instruct_peft (1).ipynb'
 output.mp4
 product_description_generator_attributes_to_text.ipynb
 sealion_test
 trim_drain_2.mp4


In [18]:

!mkdir ./books
!mkdir ./matchingengine
!mkdir ./matchingengine/embeddings

mkdir: cannot create directory ‘./books’: File exists
mkdir: cannot create directory ‘./matchingengine’: File exists
mkdir: cannot create directory ‘./matchingengine/embeddings’: File exists


### Define helper functions

In [19]:
# Function to create Document AI Processor
def create_processor(project_id, location, processor_display_name, processor_type):
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(project_id, location)

    # Create a processor
    processor = client.create_processor(
        parent=parent,
        processor=documentai.Processor(
            display_name=processor_display_name, type_=processor_type
        ),
    )
    
    processor_id = processor.name.split('/')[-1]

    # Print the processor information
    print(f"Processor Name: {processor.name}")
    print(f"Processor Display Name: {processor.display_name}")
    print(f"Processor ID: {processor_id}")
    print(f"Processor Type: {processor.type_}")
    
    
    return processor, processor_id

#Function to retrieve list of existing processors
def list_processors(project_id: str, location: str) -> None:
    processorList=[]
    
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(PROJECT_ID, location)

    # Make ListProcessors request
    processor_list = client.list_processors(parent=parent)

    # Print the processor information
    for processor in processor_list:
        print(f"Processor Name: {processor.name}")
        print(f"Processor Display Name: {processor.display_name}")
        print(f"Processor Type: {processor.type_}")
        processorList.append(processor)
        
    return processorList

### Import Document AI libraries and set variables


In [20]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai

GCP_PROJECT = PROJECT_ID #'cloud-llm-preview1'
GCP_REGION='asia-southeast1'

# Variables for Document AI OCR Processor
PROCESSOR_DISPLAY_NAME = UNIQUE_PREFIX + '-ocr-processor' # Must be unique per project, e.g.: 'My Processor'
PROCESSOR_TYPE = 'OCR_PROCESSOR' # Use fetch_processor_types to get available processor types

### Create Document AI Document OCR Processor


In [21]:
LOCATION="us"

processorList=list_processors(PROJECT_ID,LOCATION)

def createUniqueProcessor():
    if len(processorList)==0:
        PROCESSOR, PROCESSOR_ID = create_processor(PROJECT_ID, LOCATION,PROCESSOR_DISPLAY_NAME, PROCESSOR_TYPE)
        return PROCESSOR, PROCESSOR_ID
    else:
        for processor in processorList:
            if PROCESSOR_DISPLAY_NAME==processor.display_name:
                PROCESSOR_ID=processor.name.split('/')[-1]
                PROCESSOR=processor
                return PROCESSOR, PROCESSOR_ID
            else:
                try:
                    PROCESSOR, PROCESSOR_ID = create_processor(PROJECT_ID, LOCATION,PROCESSOR_DISPLAY_NAME, PROCESSOR_TYPE)
                    return PROCESSOR, PROCESSOR_ID
                except:
                    continue
                

PROCESSOR, PROCESSOR_ID = createUniqueProcessor()


print(f"Processor {PROCESSOR_ID} already exists.")

Processor Name: projects/255766800726/locations/us/processors/3a0cba6f2a623041
Processor Display Name: jingletest1-ocr-processor
Processor Type: OCR_PROCESSOR
Processor Name: projects/255766800726/locations/us/processors/2bfcbd882151f885
Processor Display Name: asia-notebooks-ocr-processor
Processor Type: OCR_PROCESSOR
Processor Name: projects/255766800726/locations/us/processors/abd8c64b34e1f461
Processor Display Name: genai_processor
Processor Type: CUSTOM_EXTRACTION_PROCESSOR
Processor Name: projects/255766800726/locations/us/processors/8563cd6b7c47e789
Processor Display Name: IMDA
Processor Type: SUMMARY_PROCESSOR
Processor Name: projects/255766800726/locations/us/processors/14d20d44606f1fa7
Processor Display Name: AGD extractor
Processor Type: CUSTOM_EXTRACTION_PROCESSOR
Processor Name: projects/255766800726/locations/us/processors/80e28448feadd169
Processor Display Name: novel-summariser
Processor Type: SUMMARY_PROCESSOR
Processor Name: projects/255766800726/locations/us/processo

In [22]:
%pip install --upgrade --quiet  google-cloud-documentai
%pip install --upgrade --quiet  google-cloud-documentai-toolbox

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/python_dateutil-2.8.2.dist-info/METADATA'

Note: you may need to restart the kernel to use updated packages.


In [23]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai


# PROJECT_ID = "YOUR_PROJECT_ID"
GCP_PROJECT= PROJECT_ID #'cloud-llm-preview1'
LOCATION = "us"  # Format is 'us' or 'eu'
PROCESSOR_ID = PROCESSOR_ID  # Create processor in Cloud Console
GCP_REGION="asia-southeast1"

# The local file in your current working directory
FILE_PATH = f"{FOLDER_NAME}/books/Book1_HarryPotter_and_the_Sorcerers_Stone_pg15.pdf"
# Refer to https://cloud.google.com/document-ai/docs/file-types
# for supported file types
MIME_TYPE = "application/pdf"

# Instantiates a client
docai_client = documentai.DocumentProcessorServiceClient(
    client_options=ClientOptions(api_endpoint=f"{LOCATION}-documentai.googleapis.com")
)

# The full resource name of the processor, e.g.:
# projects/project-id/locations/location/processor/processor-id
# You must create new processors in the Cloud Console first
RESOURCE_NAME = docai_client.processor_path(PROJECT_ID, LOCATION, PROCESSOR_ID)

# Read the file into memory
with open(FILE_PATH, "rb") as image:
    image_content = image.read()

# Load Binary Data into Document AI RawDocument Object
raw_document = documentai.RawDocument(content=image_content, mime_type=MIME_TYPE)

# Configure the process request
request = documentai.ProcessRequest(name=RESOURCE_NAME, raw_document=raw_document)

# Use the Document AI client to process the sample form
result = docai_client.process_document(request=request)

document_object = result.document
print("Document processing complete.")
print(f"Text: {document_object.text}")

page_text =document_object.text

Document processing complete.
Text: CHAPTER ONE
THE BOY WHO LIVED
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last people
you'd expect to be involved in anything strange or mysterious, because they just
didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did have a
very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the
usual amount of neck, which came in very useful as she spent so much of her
time craning over garden fences, spying on the neighbors. The Dursleys had a
small son called Dudley and in their opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't think they
could bear it if anyone found out about the Potters. Mrs. Potte

### Running though the batch mode for procssing the full Books

In [24]:
import re
from typing import Optional

from google.api_core.client_options import ClientOptions
from google.api_core.exceptions import InternalServerError
from google.api_core.exceptions import RetryError
from google.cloud import documentai  # type: ignore
from google.cloud import storage

def batch_process_documents(
    project_id: str,
    location: str,
    processor_id: str,
    gcs_output_uri: str,
    processor_version_id: Optional[str] = None,
    gcs_input_uri: Optional[str] = None,
    input_mime_type: Optional[str] = None,
    gcs_input_prefix: Optional[str] = None,
    field_mask: Optional[str] = None,
    timeout: int = 4000000000,
) -> None:
    # You must set the `api_endpoint` if you use a location other than "us".
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    if gcs_input_uri:
        # Specify specific GCS URIs to process individual documents
        gcs_document = documentai.GcsDocument(
            gcs_uri=gcs_input_uri, mime_type=input_mime_type
        )
        # Load GCS Input URI into a List of document files
        gcs_documents = documentai.GcsDocuments(documents=[gcs_document])
        input_config = documentai.BatchDocumentsInputConfig(gcs_documents=gcs_documents)
    else:
        # Specify a GCS URI Prefix to process an entire directory
        gcs_prefix = documentai.GcsPrefix(gcs_uri_prefix=gcs_input_prefix)
        input_config = documentai.BatchDocumentsInputConfig(gcs_prefix=gcs_prefix)

    # Cloud Storage URI for the Output Directory
    gcs_output_config = documentai.DocumentOutputConfig.GcsOutputConfig(
        gcs_uri=gcs_output_uri, field_mask=field_mask
    )

    # Where to write results
    output_config = documentai.DocumentOutputConfig(gcs_output_config=gcs_output_config)

    if processor_version_id:
        # The full resource name of the processor version, e.g.:
        # projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}
        name = client.processor_version_path(
            project_id, location, processor_id, processor_version_id
        )
    else:
        # The full resource name of the processor, e.g.:
        # projects/{project_id}/locations/{location}/processors/{processor_id}
        name = client.processor_path(project_id, location, processor_id)

    request = documentai.BatchProcessRequest(
        name=name,
        input_documents=input_config,
        document_output_config=output_config,
    )

    # BatchProcess returns a Long Running Operation (LRO)
    operation = client.batch_process_documents(request)

    # Continually polls the operation until it is complete.
    # This could take some time for larger files
    # Format: projects/{project_id}/locations/{location}/operations/{operation_id}
    try:
        print(f"Waiting for operation {operation.operation.name} to complete...")
        operation.result(timeout=timeout)
    # Catch exception when operation doesn't finish before timeout
    except (RetryError, InternalServerError) as e:
        print(e.message)

    # Once the operation is complete,
    # get output document information from operation metadata
    metadata = documentai.BatchProcessMetadata(operation.metadata)

    if metadata.state != documentai.BatchProcessMetadata.State.SUCCEEDED:
        raise ValueError(f"Batch Process Failed: {metadata.state_message}")

    storage_client = storage.Client()

    print("Output files:")
    # One process per Input Document
    for process in list(metadata.individual_process_statuses):
        # output_gcs_destination format: gs://BUCKET/PREFIX/OPERATION_NUMBER/INPUT_FILE_NUMBER/
        # The Cloud Storage API requires the bucket name and URI prefix separately
        matches = re.match(r"gs://(.*?)/(.*)", process.output_gcs_destination)
        if not matches:
            print(
                "Could not parse output GCS destination:",
                process.output_gcs_destination,
            )
            continue

        output_bucket, output_prefix = matches.groups()

        # Get List of Document Objects from the Output Bucket
        output_blobs = storage_client.list_blobs(output_bucket, prefix=output_prefix)

        # Document AI may output multiple JSON files per source file
        for blob in output_blobs:
            # Document AI should only output JSON files to GCS
            if blob.content_type != "application/json":
                print(
                    f"Skipping non-supported file: {blob.name} - Mimetype: {blob.content_type}"
                )
                continue

            # Download JSON File as bytes object and convert to Document Object
            print(f"Fetching {blob.name}")
            document = documentai.Document.from_json(
                blob.download_as_bytes(), ignore_unknown_fields=True
            )

            # For a full list of Document object attributes, please reference this page:
            # https://cloud.google.com/python/docs/reference/documentai/latest/google.cloud.documentai_v1.types.Document
            
            # Read the text recognition output from the processor 
            print("The document contains the following text:")
            print(document.text)
            
        return(document.text)


In [49]:
import os

def get_pdf_files(folder_path):
    """
    Reads all files in a folder and returns a list of PDF files only.
    Args:
        folder_path: The path to the folder.
    Returns:
        A list of PDF file paths.
    """

    pdf_files = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_files.append(filename)
            
    gcs_pdf_files = [f"{GCS_BUCKET_URI}/{file}" for file in pdf_files]

    return(gcs_pdf_files)

In [25]:
import asyncio
import time
import re

async def my_async_function():
    # Your asynchronous code here
    await asyncio.sleep(5)  # Placeholder for some asynchronous task
    print("Async function completed")

# Trigger the function asynchronously
async def trigger_async_function():
    await my_async_function()

asyncio.create_task(trigger_async_function())


def save_text_to_file(text, filename):
    pattern = r".*/([^/.]+)\.pdf"

    # Extract the filename
    match = re.search(pattern, filename)
    if match:
        filename = match.group(1)
        print(filename + " has been processed successfully.\n")  # Output ex: Book4_The_Goblet_of_Fire
    else:
        print("No match found")
    
    filename_txt = f'{FOLDER_NAME}/results/' + filename + ".txt"
    with open(filename_txt, 'w', encoding='utf-8') as f:
        f.write(text)

In [51]:
# PROJECT_ID = "YOUR_PROJECT_ID"
GCP_PROJECT= PROJECT_ID #'cloud-llm-preview1'
LOCATION = location = "us"  # Format is 'us' or 'eu'
project_id=PROJECT_ID
processor_id=PROCESSOR_ID

# The local file in your current working directory
FILE_PATH = f"{FOLDER_NAME}/books/Book1_The_Sorcerers_Stone.pdf"
# Refer to https://cloud.google.com/document-ai/docs/file-types
# for supported file types

# TODO(developer): Uncomment these variables before running the sample.
gcs_output_uri = f"{GCS_BUCKET_URI}/" # Must end with a trailing slash `/`. Format: gs://bucket/directory/subdirectory/
# processor_version_id = "" # Optional. Example: pretrained-ocr-v1.0-2020-09-23

# TODO(developer): You must specify either `gcs_input_uri` and `mime_type` or `gcs_input_prefix`
gcs_input_uri = f"{GCS_BUCKET_URI}/Book1_The_Sorcerers_Stone.pdf" # Format: gs://bucket/directory/file.pdf
MIME_TYPE = input_mime_type = "application/pdf"

gcs_input_prefix = f"{GCS_BUCKET_URI}/matchingengine/" # Format: gs://bucket/directory/
field_mask = "text,entities,pages.pageNumber"  # Optional. The fields to return in the Document object.
timeout = 400000

folder_path = "./genai_03/books/"

book_list = get_pdf_files(folder_path)
print(book_list)

# book_list = [f"{GCS_BUCKET_URI}/Winnie_the_Pooh_3_Pages.pdf",
#              f"{GCS_BUCKET_URI}/Book2_The_Chamber_of_Secrets.pdf",
#              f"{GCS_BUCKET_URI}/Book3_The_Prisoner_of_Azkaban.pdf",
#              f"{GCS_BUCKET_URI}/Book4_The_Goblet_of_Fire.pdf",
#              f"{GCS_BUCKET_URI}/Book5_The_Order_of_the_Phoenix.pdf",
#              f"{GCS_BUCKET_URI}/Book6_The_HalfBlood_Prince.pdf",
#              f"{GCS_BUCKET_URI}/Book7_The_Deathly_Hallows.pdf",]


try: 
    for i in range(0,len(book_list)): 
        gcs_input_uri = book_list[i]
        print(gcs_input_uri + ":\n")

        page_text_batch = batch_process_documents(   project_id,
            location,
            processor_id,
            gcs_output_uri,
            None,
            gcs_input_uri,
            input_mime_type,
            gcs_input_prefix,
            field_mask,
            timeout)

        # Example usage: my_text = "This is the text from your OCR process."

        save_text_to_file(page_text_batch, gcs_input_uri)
        time.sleep(0.1)
except :
    print("""Run this again - click on restart kernel and Run everything if you want to test doc AI part 
          --- Here are the steps :
          1. Look at the menu and go to kernel
          2. Click the 5th option Restart Kernel and run everything 
          """)

['gs://my-project-0004-346516-singhealth/Winnie_the_Pooh_3_Pages.pdf', 'gs://my-project-0004-346516-singhealth/Book1_HarryPotter_and_the_Sorcerers_Stone_pg15.pdf']
gs://my-project-0004-346516-singhealth/Winnie_the_Pooh_3_Pages.pdf:

Waiting for operation projects/255766800726/locations/us/operations/8136127822156390750 to complete...
Output files:
Fetching 8136127822156390750/0/Winnie_the_Pooh_3_Pages-0.json
The document contains the following text:
CHAPTER I
IN WHICH We Are Introduced to
Winnie-the-Pooh and Some
Bees, and the Stories Begin
HERE is Edward Bear, coming
downstairs now, bump, bump, bump, on the back
of his head, behind Christopher Robin. It is, as far
as he knows, the only way of coming downstairs,
but sometimes he feels that there really is another
way, if only he could stop bumping for a moment
and think of it. And then he feels that perhaps there
isn't. Anyhow, here he is at the bottom, and ready
to be introduced to you. Winnie-the-Pooh.
When I first heard his name, I 

## Done